## Install Packages

In [11]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import zipfile 
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from sklearn.metrics import accuracy_score
import src
from src.utils.celeba_helper import CelebADataset, CelebAClassifier, save_file_names
from imp import reload

workers = 0 if os.name == 'nt' else 2

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Define CelebA Dataset and Loader

In [13]:
## Load the dataset
# Path to directory with all the images
img_folder = 'data/img_align_celeba'
mapping_file = 'data/identity_CelebA.txt'

# Spatial size of training images, images are resized to this size.
image_size = 160
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor()
])

# Load the dataset from file and apply transformations
celeba_dataset = CelebADataset(img_folder, mapping_file, transform)

In [14]:
## Create a dataloader
# Batch size during training
batch_size = 128
# Number of workers for the dataloader
num_workers = 0 if device.type == 'cuda' else 2
# Whether to put fetched data tensors to pinned memory
pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = torch.utils.data.DataLoader(celeba_dataset,  # type: ignore
                                                batch_size=batch_size,
                                                num_workers=num_workers,
                                                pin_memory=pin_memory,
                                                shuffle=False)

# Setup FaceNet

In [15]:
mtcnn = MTCNN(
    image_size=image_size, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, keep_all=False,
    device=device
)

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [16]:
classifier = CelebAClassifier(celeba_dataloader, detection_model=mtcnn, embedding_model=resnet)

# One-Shot Learning


## Load Dataset

In [17]:
# Select train files
file_label_mapping = celeba_dataset.get_file_label_mapping()
first_file_for_each_person_df = file_label_mapping.sort_values(by='person_id').groupby('person_id').agg(['min', 'count'])
train_files = np.sort(first_file_for_each_person_df[first_file_for_each_person_df['file_name']['count'] > 1]['file_name']['min'].values)

In [18]:
# The following takes a long time, as one image is loaded and embedded for each person. Run this once for the desired dataset. 
# Afterwards, load the data from the pickle file.

if not os.path.exists('pytorch_objects'):
  os.makedirs('pytorch_objects')

embeddings_path = 'pytorch_objects/train_embeddings_all_1img.pickle'
file_names_path = 'pytorch_objects/train_file_names_all_1img'

if not os.path.exists(embeddings_path) or not os.path.exists(file_names_path):
    train_embeddings, train_face_file_names = classifier.load_data_specific_images(files_to_load=train_files)
    torch.save(train_embeddings, f'pytorch_objects/train_embeddings_all_1img.pickle')
    save_file_names(train_face_file_names, 'pytorch_objects/train_file_names_all_1img')
else:
    train_embeddings = torch.load(embeddings_path)
    train_face_file_names = []
    with open(file_names_path, 'r') as fp:
        for line in fp:
            x = line[:-1]
            # add current item to the list
            train_face_file_names.append(x)

print(f'Size of train dataset: {train_embeddings.shape}')
train_labels = celeba_dataset.get_labels_from_file_names(train_face_file_names)

Size of train dataset: torch.Size([10115, 512])
Number of people in dataset: 10115


In [19]:
# Select test files
second_file_for_each_person_df = file_label_mapping[~file_label_mapping['file_name'].isin(first_file_for_each_person_df['file_name']['min'])].sort_values(by='person_id').groupby('person_id').agg(['min', 'count'])
test_files = np.sort(second_file_for_each_person_df[second_file_for_each_person_df['file_name']['count'] >= 1]['file_name']['min'].values)

In [20]:
# The following takes a long time, as one image is loaded and embedded for each person. Run this once for the desired dataset. 
# Afterwards, load the data from the pickle file.

embeddings_path = 'pytorch_objects/test_embeddings_all_1img.pickle'
file_names_path = 'pytorch_objects/test_file_names_all_1img'

if not os.path.exists(embeddings_path) or not os.path.exists(file_names_path):
    test_embeddings, test_face_file_names = classifier.load_data_specific_images(files_to_load=test_files)
    torch.save(test_embeddings, f'pytorch_objects/test_embeddings_all_1img.pickle')
    save_file_names(test_face_file_names, 'pytorch_objects/test_file_names_all_1img')
else:
    test_embeddings = torch.load(embeddings_path)
    test_face_file_names = []
    with open(file_names_path, 'r') as fp:
        for line in fp:
            x = line[:-1]
            # add current item to the list
            test_face_file_names.append(x)
            
print(f'Size of test dataset: {test_embeddings.shape}')
test_labels = celeba_dataset.get_labels_from_file_names(test_face_file_names)

Size of test dataset: torch.Size([10117, 512])
Number of people in dataset: 10117


## Predicting

The following cells, each might take longer.

In [11]:
for similarity_metric in ['norm_2', 'norm_2_squared', 'cosine_similarity']:
    test_predictions, test_predictions_files = classifier.predict(test_embeddings, train_embeddings, train_face_file_names, similarity_metric)
    accuracy = accuracy_score(test_labels, test_predictions)
    print(f'Accuracy - {similarity_metric}: {np.round(accuracy, 4)}')

Calculating the norm_2 metric...


10117it [24:21,  6.92it/s]


Accuracy - norm_2: 0.6955
Calculating the norm_2_squared metric...


10117it [23:22,  7.21it/s]


Accuracy - norm_2_squared: 0.6955
Calculating the cosine_similarity metric...


10117it [53:05,  3.18it/s]

Accuracy - cosine_similarity: 0.6955


In [21]:
# fit Support Vector Classifier
from sklearn.svm import SVC
model = SVC(kernel='linear', verbose=True)
model.fit(train_embeddings, train_labels)

SVC(kernel='linear')

In [ ]:
# train_predictions = model.predict(train_embeddings)
test_predictions = model.predict(test_embeddings)
# train_predictions = model.predict(train_embeddings)

# score_train = accuracy_score(train_labels, train_predictions)
score_test = accuracy_score(test_labels, test_predictions)
# score_train = accuracy_score(train_labels, train_predictions)

# print(f'Accuracy: train = {np.round(score_train*100, 3)}%')
print(f'Accuracy: test = {np.round(score_test*100, 3)}%')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(train_embeddings[0:1000], train_labels[0:1000])

knn.score(test_embeddings[0:1000], test_labels[0:1000])